# Validating values in NL0000

In [1]:
path = './Data/'

import pandas as pd
import numpy as np

## Insight 

In [2]:
node_arc_grants = pd.read_csv(path + 'node_arc_grants.csv', sep = '|', header = 0)
grant_to_arc_org = pd.read_csv(path + 'grant_to_arc_org.csv', sep = ',', header = 0)
node_arc_orgs = pd.read_csv(path + 'node_arc_orgs.csv', sep = ',', header = 0)


In [3]:
print(node_arc_grants.dtypes)
print('------------------------------------------------------------')
print(grant_to_arc_org.dtypes)
print('------------------------------------------------------------')
print(node_arc_orgs.dtypes)

key:ID                                          object
source                                          object
local_id                                        object
last_updated                                    object
url                                             object
title                                           object
purl                                            object
participant_list                                object
funder                                          object
start_year                                     float64
end_year                                       float64
arc_announcement_administering_organisation     object
arc_scheme_name                                 object
arc_grant_status                                object
arc_for                                         object
arc_rfcd                                        object
arc_funding_at_announcement                      int64
source_file                                     object
type      

In [4]:
node_arc_grants.rename(columns = {'key:ID' : 'key'}, inplace = True)
grant_to_arc_org.rename(columns = {'arc_grant_key' : 'key'}, inplace = True)

node_arc_grants['key'] = node_arc_grants['key'].astype(str)
grant_to_arc_org['key'] = grant_to_arc_org['key'].astype(str)

left_table = node_arc_grants.merge(grant_to_arc_org, on = 'key', how = 'inner')
#left_table = node_arc_grants.join(grant_to_arc_org, on = 'key', how = 'inner')

In [5]:
node_arc_orgs.rename(columns = {'key:ID' : 'arc_org_key'}, inplace = True)
node_arc_orgs['arc_org_key'] = node_arc_orgs['arc_org_key'].astype(str)

table_for_insight = left_table.merge(node_arc_orgs, on = 'arc_org_key', how = 'inner')

In [6]:
table_for_insight_mini = table_for_insight[['key', 'name']]
table_for_insight_mini_non_uni = table_for_insight[table_for_insight['name'].str.contains('University', 
                                                                                     case = False) == False]

insight = (len(table_for_insight_mini_non_uni) / len(table_for_insight_mini)) * 100

print("Percentage of partners that are not Universities : ",round(insight))

Percentage of partners that are not Universities :  31


## Number of Grants

In [7]:
table_for_grants = table_for_insight[table_for_insight['name'].str.contains('University',
                                                         case = False) == False]

print("Number of Non-University Grants from node_arc_args :", len(np.unique(np.array(table_for_grants['key']))))

Number of Non-University Grants from node_arc_args : 10543


In [8]:
grants = pd.read_csv(path + 'Grant.csv', sep = ',')

list_grants_Grant_csv = list(grants['key'])
list_grants_node_arc_orgs_csv = list(table_for_grants['key'])

list_missing_grants = []

for key in list_grants_node_arc_orgs_csv:
    if key not in list_grants_Grant_csv:
        list_missing_grants.append(key)
        
#print(list_missing_grants)

left_table_from_grant_csv = grants.merge(grant_to_arc_org, on = 'key', how = 'inner')
table_for_grants = left_table_from_grant_csv.merge(node_arc_orgs, on = 'arc_org_key', how = 'inner')

table_for_grants_non_uni = table_for_grants[table_for_grants['name'].str.contains('University',
                                                                                 case = False) == False]
print("Number of non University grants from Grants.csv file :",len(np.array(table_for_grants_non_uni['key'].unique())))

Number of non University grants from Grants.csv file : 3967


## Number of Admin Organisations

In [9]:
table_for_admin_orgs = table_for_insight[table_for_insight['name'].str.contains('University',
                                                         case = False) == False]

print("Number of Administering organisations from node_arc_grants :",
              len(np.array(table_for_admin_orgs['arc_announcement_administering_organisation'].unique())))

Number of Administering organisations from node_arc_grants : 109


In [10]:
list_admin_orgs_Grant_csv = list(grants['announcement-administering-organisation'])
list_admin_orgs_node_arc_orgs_csv = list(table_for_admin_orgs['arc_announcement_administering_organisation'])

list_missing_admin_orgs = []

for key in list_admin_orgs_node_arc_orgs_csv:
    if key not in list_admin_orgs_Grant_csv:
        list_missing_admin_orgs.append(key)
        
#print(list_missing_grants)

left_table_from_grant_csv = grants.merge(grant_to_arc_org, on = 'key', how = 'inner')
table_for_grants = left_table_from_grant_csv.merge(node_arc_orgs, on = 'arc_org_key', how = 'inner')

table_for_grants_non_uni = table_for_grants[table_for_grants['name'].str.contains('University',
                                                                                 case = False) == False]
print("Number of Administering organisations from Grants.csv file :",
          len(np.array(table_for_grants_non_uni['announcement-administering-organisation'].unique())))

Number of Administering organisations from Grants.csv file : 46


## Number of AUS organisation

In [11]:
node_abr_orgs = pd.read_csv(path + 'node_abr_orgs.csv', sep = '|', header = 0)
node_abr_orgs_non_uni = node_abr_orgs[node_abr_orgs['name'].str.contains('University', case = False) == False]

print("Number of Aus Organisation :",len(np.array(node_abr_orgs_non_uni['name'].unique())))

Number of Aus Organisation : 5955


## Partner Organisations

In [36]:
table_for_insight_non_uni = table_for_insight[table_for_insight['name'].str.contains('University', 
                                                                                    case = False) == False]
table_for_insight_partner_orgs = table_for_insight_non_uni[['key', 'name']]
print("This data is from node_arc_grants.csv")
print(table_for_insight_partner_orgs.groupby(['name']).nunique().sort_values(['key'], ascending = False))

This data is from node_arc_grants.csv
                                                    key
name                                                   
Commonwealth scientific and industrial research...  513
Other                                               431
Australian nuclear science and technology organ...  121
The walter and eliza hall institute of medical ...  100
South australian museum                              88
...                                                 ...
Goldfields aboriginal language centre                 1
Goldstar resources nl                                 1
Goldwind australia pty ltd                            1
Gollings photography pty. ltd                         1
Landcare research new zealand limited                 1

[12657 rows x 1 columns]


In [37]:
table_for_grants_non_uni = table_for_grants[table_for_grants['name'].str.contains('University', 
                                                                                    case = False) == False]
table_for_grants_partner_orgs = table_for_grants_non_uni[['key', 'name']]
print("This data is from Grants.csv")
print(table_for_grants_partner_orgs.groupby(['name']).nunique().sort_values(['key'], ascending = False))

This data is from Grants.csv
                                                    key
name                                                   
Commonwealth scientific and industrial research...  291
Australian nuclear science and technology organ...   75
Imperial college london                              63
Massachusetts institute of technology                51
Melbourne water corporation                          50
...                                                 ...
H.j. heinz company australia limited                  1
Gydle inc                                             1
Gwydir shire council                                  1
Guodian new energy technology research institute      1
Landgate                                              1

[5458 rows x 1 columns]


In [32]:
table_for_grants.dtypes

code                                        object
key                                         object
announcement-administering-organisation     object
scheme-name                                 object
funding-at-announcement                      int64
grant-status                                object
funding-commencement-year                    int64
project-start-date                          object
anticipated-end-date                        object
funding-current                            float64
grant_title                                 object
arc_org_key                                 object
source                                      object
local_id                                    object
last_updated                                object
name                                        object
source_file                                 object
type                                        object
:LABEL                                      object
dtype: object

In [13]:
print("Number of non university grants from node_arc_grants.csv :",
              len(np.array(table_for_insight_non_uni['key'].unique())))

Number of non university grants from node_arc_grants.csv : 10543


In [35]:
print('CSIRO :', 513/10543)
print('CSIRO grant.csv :', 291/10543)

CSIRO : 0.04865787726453571
CSIRO grant.csv : 0.027601252015555345


## Australian Business Register Organisations

In [15]:
node_abr_orgs_non_uni.groupby(['state'])['key:ID'].nunique()

state
         56
ACT     222
NSW    1886
NT       86
QLD     899
SA      468
TAS     128
VIC    1625
WA      589
Name: key:ID, dtype: int64

## Keyword Frequency in Grant Summary

In [16]:
grant_summary = pd.read_csv(path + 'grant_summary(v2).csv', sep = ',', header = 0)

table_for_keyword_summary = table_for_grants_non_uni.merge(grant_summary, on = 'code', how = 'inner')

In [17]:
table_for_keyword_summary = table_for_keyword_summary[['key', 'keywords_by_gensim']]
table_for_keyword_summary.groupby(['keywords_by_gensim']).nunique().sort_values(['key'], ascending = False)

#This visual has a filter which filters out the keyword 'project'.

,key
keywords_by_gensim,
project,1761
australia,261
research,259
technology,234
industry,171
...,...
hunter,1
hyaluronidase,1
hybrid,1


## Fields of Research

In [18]:
for_graph = pd.read_csv(path + 'FOR_graph.csv', sep = ',', header = 0)
grant_for = pd.read_csv(path + 'grant_for.csv', sep = ',', header = 0)

grant_for.rename(columns = {'Code' : 'code'}, inplace = True)
for_graph['key'] = for_graph['Type'].map(str) + ':' + '0' + for_graph['TargetCode'].map(str)

left_table_field_research = table_for_grants_non_uni.merge(grant_for, on = 'code', how = 'inner')
left_table_field_research.rename(columns = {'key_y' : 'key'}, inplace = True)

table_for_field_research = left_table_field_research.merge(for_graph, on = 'key', how = 'inner')

In [19]:
table_for_field_research = table_for_field_research[['key_x', 'TargetFOR']]
table_for_field_research.groupby(['TargetFOR']).nunique().sort_values(['key_x'], ascending = False)

,key_x
TargetFOR,
Environmental Science and Management,282
Materials Engineering,275
Biochemistry and Cell Biology,259
Civil Engineering,195
Artificial Intelligence and Image Processing,189
...,...
Other Agricultural and Veterinary Sciences,5
Other Information and Computing Sciences,4
Other Earth Sciences,4


## FOR Codes

In [20]:
print("Number of FOR Codes :",len(np.array(left_table_field_research['key'].unique())))

Number of FOR Codes : 175


## Commencement Year

In [21]:
table_for_insight_non_uni.groupby(['start_year'])['key'].nunique()

start_year
2000.0      2
2001.0     30
2002.0    502
2003.0    593
2004.0    553
2005.0    540
2006.0    466
2007.0    430
2008.0    462
2009.0    513
2010.0    903
2011.0    791
2012.0    605
2013.0    446
2014.0    536
2015.0    437
2016.0    554
2017.0    351
2018.0    286
2019.0    320
2020.0    361
2021.0    436
2022.0    404
2023.0      9
Name: key, dtype: int64

In [22]:
table_for_grants_non_uni.groupby(['funding-commencement-year'])['key'].nunique()

funding-commencement-year
2013    515
2014    504
2015    447
2016    420
2017    309
2018    326
2019    328
2020    455
2021    384
2022    279
Name: key, dtype: int64

## Socio Economic Objective by number of collaborative Grants

In [23]:
socio_economic_objective = pd.read_csv(path + 'socio_economic_objective.csv', sep = ',', header = 0)
socio_economic_objective.rename(columns = {'Grant Code' : 'code', 'Code' : 'key'}, inplace = True)

table_for_socio_economic_objective = table_for_grants_non_uni.merge(socio_economic_objective,
                                                           on = 'code', how = 'inner')

In [24]:
table_for_socio_economic_objective = table_for_socio_economic_objective[['Name', 'key_x']]
table_for_socio_economic_objective.groupby(['Name']).nunique().sort_values(['key_x'], ascending = False)

,key_x
Name,
Expanding Knowledge In the Biological Sciences,489
Expanding Knowledge In the Physical Sciences,369
Expanding Knowledge In Engineering,308
Expanding Knowledge In Technology,264
Expanding Knowledge In the Chemical Sciences,206
...,...
Management of Greenhouse Gas Emissions From Animal Production,1
Soy Products,1
Sown Pastures (Excl. Lucerne),1


## Funding Scheme by Number of Collaborative Grants 

In [27]:
table_for_funding_scheme_non_uni = table_for_insight_non_uni[['arc_scheme_name', 'key']]
print("This data is from node_arc_orgs.csv")
print(table_for_funding_scheme_non_uni.groupby(['arc_scheme_name']).nunique().sort_values(['key'], ascending = False))

This data is from node_arc_orgs
                                                     key
arc_scheme_name                                         
Linkage Projects                                    6893
Discovery Projects                                  2340
Linkage Infrastructure, Equipment and Facilities     580
ARC Future Fellowships                               319
ARC Centres of Excellence                             79
Industrial Transformation Training Centres            63
Learned Academies Special Projects                    59
Industrial Transformation Research Hubs               51
Special Research Initiatives                          43
Discovery Indigenous                                  30
Research Networks                                     21
Super Science Fellowships                             20
Discovery Early Career Researcher Award               14
Federation Fellowships                                10
Linkage - CSIRO                                       10

In [26]:
table_for_funding_scheme_non_uni_non_Grant = table_for_grants_non_uni[['scheme-name', 'key']]
print("This data is from Grants.csv")
print(table_for_funding_scheme_non_uni_non_Grant.groupby(['scheme-name']).\
                                                    nunique().sort_values(['key'], ascending = False))

,key
scheme-name,
Linkage Projects,1953
Discovery Projects,1468
"Linkage Infrastructure, Equipment and Facilities",235
ARC Future Fellowships,113
Industrial Transformation Training Centres,63
Industrial Transformation Research Hubs,47
Special Research Initiatives,30
ARC Centres of Excellence,29
Discovery Indigenous,26
